---
title: 4 Coupled KPOs
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.4
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
# from tqdm import tqdm
from qutip.ui.progressbar import BaseProgressBar, TextProgressBar

## Parameters

## $\hbar = 1$

In [27]:
N = 2 # Number of KPOs
n_levels = 5 # Truncation level for Fock space
K = 1.0 # Kerr nonlinearity
p = 7.0 * K # Maximum pump strength
xi = 0.5 * K # Coupling constant
T = 700 / K # Total evolution time
num_steps = 1000 # number of time steps

## Symmetric coupling matrix J

In [28]:
J = np.random.uniform(-1, 1, (N, N))
J = (J + J.T) / 2 # Make J symmetric
np.fill_diagonal(J, 0) # Ensure no self coupling
J = Qobj(J)

In [29]:
J

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.         -0.68617648]
 [-0.68617648  0.        ]]

## Annihilation operator $\hat{a}_{i}$

In [30]:
a = [destroy(n_levels) for _ in range(N)]

In [31]:
a[0]

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dia, isherm=False
Qobj data =
[[0.         1.         0.         0.         0.        ]
 [0.         0.         1.41421356 0.         0.        ]
 [0.         0.         0.         1.73205081 0.        ]
 [0.         0.         0.         0.         2.        ]
 [0.         0.         0.         0.         0.        ]]

## Adiabatic evolution condition
## $\Delta_{i} = \xi_{0} \sum_{j=1}^{N} |J_{i,j}|$

In [32]:
Delta = [xi * np.sum(np.abs(J[i])) for i in range(N)]

In [33]:
Delta

[0.343088237588262, 0.343088237588262]

## Single KPO Hamiltonian terms

### $H_{\Delta}^{(i)} = \Delta_{i} a^\dagger_{i} a_{i}$

In [34]:
H_D = [Delta[i] * a[i].dag() * a[i] for i in range(N)]

### $H_{K}^{(i)} = \frac{K}{2} {a^\dagger_{i}}^{2} {a_{i}}^{2}$

In [35]:
H_K = [(K / 2) * a[i].dag() * a[i].dag() * a[i] * a[i] for i in range(N)]

In [36]:
H_K[0]

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dia, isherm=True
Qobj data =
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 6.]]

### $H_{p}^{(i)} = {a^\dagger_{i}}^{2} + {a_{i}}^{2}$
### $\frac{p}{2}$ will be multiplied to this term during time evolution

In [37]:
H_plist = [a[i].dag() * a[i].dag() + a[i] * a[i] for i in range(N)]

In [38]:
H_plist[0]

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dia, isherm=True
Qobj data =
[[0.         0.         1.41421356 0.         0.        ]
 [0.         0.         0.         2.44948974 0.        ]
 [1.41421356 0.         0.         0.         3.46410162]
 [0.         2.44948974 0.         0.         0.        ]
 [0.         0.         3.46410162 0.         0.        ]]

## Coupling term
## $H_{C} = \frac{\xi_{0}}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} J_{i,j} ( a_{i}^\dagger a_{j} + a_{i} a_{j}^\dagger)$ 

In [39]:
H_C = 0.5 * xi * sum([J[i, j] * (a[i].dag() * a[j] + a[i] * a[j].dag()) 
                      for i in range(N) for j in range(N)])

In [40]:
H_C

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dia, isherm=True
Qobj data =
[[-0.34308824  0.          0.          0.          0.        ]
 [ 0.         -1.02926471  0.          0.          0.        ]
 [ 0.          0.         -1.71544119  0.          0.        ]
 [ 0.          0.          0.         -2.40161766  0.        ]
 [ 0.          0.          0.          0.         -1.37235295]]

## Identity operator

In [41]:
I = qeye(n_levels)

In [42]:
I

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dia, isherm=True
Qobj data =
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

## Time-dependent Hamiltonian
## $H(P(t))$

In [43]:
def H_t(t, args):
    p = args['p_max'] * (t / T)
    H_p = (p / 2) * sum([tensor([H_plist[i] if i == k else I for k in range(N)]) for i in range(N)])
    H = sum([tensor([H_K[i] if i == k else I for k in range(N)]) for i in range(N)]) + \
        sum([tensor([H_D[i] if i == k else I for k in range(N)]) for i in range(N)]) - \
        tensor([H_C if i == 0 else qeye(n_levels) for i in range(N)])

    return H - H_p

## Arguments for TDH

In [44]:
H_args = {'p_max': p}

In [45]:
H_args

{'p_max': 7.0}

## Initial state $\ket{0} \otimes \ket{0} \otimes \ket{0} \otimes \ket{0}$

In [46]:
psi0 = tensor([basis(n_levels, 0) for _ in range(N)])

In [47]:
psi0

Quantum object: dims=[[5, 5], [1, 1]], shape=(25, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

## Time evolution

In [48]:
times = np.linspace(0, T, num_steps + 1)

In [49]:
result = mesolve(H_t, psi0, times, [], [], args=H_args, options={"progress_bar": "tqdm"})

ModuleNotFoundError: No module named 'tqdm'

### Extract final state

In [ ]:
final_state = result.states[-1]
final_state

Quantum object: dims=[[5, 5], [1, 1]], shape=(25, 1), type='ket', dtype=Dense
Qobj data =
[[0.06033036-0.11155821j]
 [0.        +0.j        ]
 [0.12436617-0.22457515j]
 [0.        +0.j        ]
 [0.09109157-0.1637439j ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.136019  -0.24569629j]
 [0.        +0.j        ]
 [0.28044677-0.49447726j]
 [0.        +0.j        ]
 [0.20518078-0.36060378j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.10006538-0.17999859j]
 [0.        +0.j        ]
 [0.20609533-0.36232123j]
 [0.        +0.j        ]
 [0.15102825-0.2641402j ]]

### The complex values arise because the system accumulates a global phase factor over the adiabatic evolution process

In [ ]:
fs = Qobj(np.abs(final_state.full()))
fs

Quantum object: dims=[[25], [1]], shape=(25, 1), type='ket', dtype=Dense
Qobj data =
[[0.12682659]
 [0.        ]
 [0.25671178]
 [0.        ]
 [0.18737593]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.28083418]
 [0.        ]
 [0.56847   ]
 [0.        ]
 [0.41489064]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.20594313]
 [0.        ]
 [0.41683565]
 [0.        ]
 [0.30426893]]

## Ground state of final Hamiltonian

In [ ]:
H_f = H_t(T, args=H_args)

In [ ]:
H_f

Quantum object: dims=[[5, 5], [5, 5]], shape=(25, 25), type='oper', dtype=Dia, isherm=True
Qobj data =
[[ -0.16269808   0.          -4.94974747   0.           0.
    0.           0.           0.           0.           0.
   -4.94974747   0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.          -8.5732141    0.
    0.           0.           0.           0.           0.
    0.          -4.94974747   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [ -4.94974747   0.           1.16269808   0.         -12.12435565
    0.           0.           0.           0.           0.
    0.           0.          -4.94974747   0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.

In [ ]:
ground_state = H_f.groundstate()[1]

In [ ]:
gs = Qobj(np.abs(ground_state.full()))
gs

Quantum object: dims=[[25], [1]], shape=(25, 1), type='ket', dtype=Dense
Qobj data =
[[1.32653597e-01]
 [3.33066907e-16]
 [2.67291971e-01]
 [3.33066907e-16]
 [1.94939575e-01]
 [0.00000000e+00]
 [0.00000000e+00]
 [8.67361738e-19]
 [9.23265913e-20]
 [2.84492802e-19]
 [2.81107579e-01]
 [2.84528250e-16]
 [5.66421119e-01]
 [2.13517961e-16]
 [4.13098424e-01]
 [8.21996092e-20]
 [8.43077334e-20]
 [3.24271556e-19]
 [5.71961578e-20]
 [2.15039135e-19]
 [2.04677985e-01]
 [1.71959461e-16]
 [4.12418382e-01]
 [1.34923728e-16]
 [3.00782189e-01]]

### Check if the final state after adiabatic evolution is equal to the instantaneous ground state of the final hamiltonian

In [ ]:
def cossim(fs, gs):
    if np.linalg.norm(fs) * np.linalg.norm(gs) != 0:
        cossim = np.dot(fs.T, gs) / (np.linalg.norm(fs) * np.linalg.norm(gs))
        return cossim.item()  # Convert the 2D array to a scalar
    else:
        return 0  # Return 0 as a scalar

sim = np.real(cossim(fs.full(), gs.full()))

0.9998780710307189

In [ ]:
H_i_eigen_val, H_i_Eigenvecs = H_t(0, args=H_args).eigenstates()
print(H_i_eigen_val)

NameError: name 'H_t' is not defined